**Portrait Video Segmentation Using Custom Medaipipe Calculator**

In this demo, we will build a portrait segmentaion aplication using **custom calculators** on **desktop**, using mediapipe. There will be two video file inputs and one video fiile output. Our aim is to **blend** the portrait foreground region into the background video, with the help of segmentaion **mask**. As in the case of android, we will follow the basic segmentation pipeline from **hair segmentaion** example. Since the application uses **gpu** operations, choose a GPU runtime for development and deployment.

**1. Checkout MediaPipe Github Repository**

The mediapipe **repository** contains many demo applications for android. We will modify the hair_segmentaion application, which contains the basic pipeline for **video segmentaion**.

In [ ]:
!git clone https://github.com/google/mediapipe.git
%cd mediapipe
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel

**3. Install a JDK (optional)**

Sometimes the **default jdk** version may cause an **error** during android sdk installation in ubuntu. So, install an older version of **openjdk-8** and configure the same as the default version of the system.

In [ ]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java  # Choose OpenJDK 8
!java -version

**3. Install OpenCV (optional)**

If opencv is not already installed, run **setup_opencv.sh** to automatically build OpenCV from source and modify MediaPipe’s OpenCV config.

In [ ]:
!bash setup_opencv.sh

**4. Install MediaPipe Without Android Studio (SDK & NDK)**

If the android studio is not installed in your system, you can configute mediapipe with **sdk and ndk** by running this script.

In [ ]:
!bash setup_android_sdk_and_ndk.sh

**Note:** If Android SDK and NDK are already installed, set **ANDROID_HOME** and **ANDROID_NDK_HOME** paths accordingly.

```
export ANDROID_HOME=<path to the Android SDK>
export ANDROID_NDK_HOME=<path to the Android NDK>
```

**6. Modify The Hair Segmentaion Mediapipe Application**

Initially put the **[portrait_segmentation](https://github.com/anilsathyan7/Portrait-Segmentation/blob/master/mediapipe_slimnet/portrait_segmentation.tflite)** tflite file into **models** directory, inside mediapipe folder.

**A.** Create a new directory called **portrait_segmentation** under **graphs** subdirectory and copy all the files from **hair_segmentation**.

In [4]:
!mkdir mediapipe/graphs/portrait_segmentation
!cp -r mediapipe/graphs/hair_segmentation/* mediapipe/graphs/portrait_segmentation
!mv mediapipe/graphs/portrait_segmentation/hair_segmentation_mobile_gpu.pbtxt mediapipe/graphs/portrait_segmentation/portrait_segmentation_mobile_gpu.pbtxt

Rename the pbtxt file(mv command) as **portrait_segmentation_mobile_gpu.pbtxt** and modify the following lines :-

1. **Number of channels**: 'max_num_channels: 4' in **TfLiteConverterCalculator** to max_num_channels: 3
2. **Model name**: 'hair_segmentaion.tflite' in **TfLiteInferenceCalculator** to portrait_segmentation.tflite


**B**. Add two new nodes for video file input i.e **OpenCvVideoDecoderCalculator** and one node for video file output i.e **OpenCvVideoEncoderCalculator** in the new pipeline.

```
# Decodes an input video file into images and a video header.
node {
  calculator: "OpenCvVideoDecoderCalculator"
  input_side_packet: "INPUT_FILE_PATH:input_video_path"
  output_stream: "VIDEO:input_video"
  output_stream: "VIDEO_PRESTREAM:input_video_header"
}

# Decodes an input video file into images and a video header.
node {
  calculator: "OpenCvVideoDecoderCalculator"
  input_side_packet: "INPUT_FILE_PATH:side_video_path"
  output_stream: "VIDEO:side_video"
  output_stream: "VIDEO_PRESTREAM:side_video_header"
}

# Encodes the annotated images into a video file, adopting properties specified
# in the input video header, e.g., video framerate.
node {
  calculator: "OpenCvVideoEncoderCalculator"
  input_stream: "VIDEO:output_video"
  input_stream: "VIDEO_PRESTREAM:input_video_header"
  input_side_packet: "OUTPUT_FILE_PATH:output_video_path"
  node_options: {
    [type.googleapis.com/mediapipe.OpenCvVideoEncoderCalculatorOptions]: {
      codec: "avc1"
      video_format: "mp4"
    }
  }
}
```

**Note:** The main input should be a video file contaning portrait images and the other one should be a background video.

**C.** Now remove the 'RecolorCalculator' node, and instead add the custom **SeamlessCloningCalculator**  into the pipeline.

```
# Takes Image, Mask and Background as input and performs 
# poisson blending using opencv library
node {
  calculator: "SeamlessCloningCalculator"
  input_stream: "IMAGE_CPU:input_video"
  input_stream: "BACKGROUND_CPU:sync_side_video"
  input_stream: "MASK_CPU:portrait_mask_cpu"
  output_stream: "OUTPUT_VIDEO:output_video"
}
```
**Note:** The idea is to combine the foreground in the image with background using mask, such that portrait foreground blends into the background image seamlessly.

**D**. Use **PacketClonerCalculator** to clone the background video frames when all frames are used up. Also use **ImageFrameToGpuBufferCalculator** and **GpuBufferToImageFrameCalculator** for copying data between CPU and GPU, whenever necessary.

**E**. Now, inside the **BUILD** file in this directory(portrait_segmentation), change the graph name to "**portrait_segmentation_mobile_gpu.pbtxt**".

Also add the  **calculator files** inside the **cc_library** section for mobile_calcualators as follows:-

```
"//mediapipe/calculators/video:opencv_video_decoder_calculator",
"//mediapipe/calculators/video:opencv_video_encoder_calculator",
"//mediapipe/calculators/image:poisson_blending_calculator",
"//mediapipe/calculators/core:packet_cloner_calculator",
```

See the final pbtxt file: [portrait_segmentation_mobile_gpu.pbtxt](https://github.com/anilsathyan7/Portrait-Segmentation/blob/master/mediapipe_slimnet/desktop/portrait_segmentation_mobile_gpu.pbtxt)


**F**. Similary, create a new folder called '**portrait_segmentation**' inside example directory at location: '**/mediapipe/examples/desktop/**'.Add the BUILD file  inside this directory as shown below.

```
licenses(["notice"])
package(default_visibility = ["//mediapipe/examples:__subpackages__"])

# Linux only
cc_binary(
    name = "portrait_segmentation_gpu",
    deps = [
        "//mediapipe/examples/desktop:simple_run_graph_main",
        "//mediapipe/examples/desktop:demo_run_graph_main_gpu",
        "//mediapipe/graphs/portrait_segmentation:mobile_calculators",
    ],
)
```
**Note:** We added a dependency '**simple_run_graph_main**' for executing graph using side packets and video file inputs.

**G.** Add the following dependencies into the **BUILD** file inside **calculators/image** directory.

```
cc_library(
    name = "poisson_blending_calculator",
    srcs = ["poisson_blending_calculator.cc"],
    visibility = ["//visibility:public"],    
    deps = [
        "//mediapipe/gpu:gl_calculator_helper",
        "//mediapipe/framework:calculator_framework",
        "//mediapipe/framework:calculator_options_cc_proto",
        "//mediapipe/framework:timestamp",
        "//mediapipe/framework/port:status",
        "//mediapipe/framework/deps:file_path",
        "@com_google_absl//absl/time",
        "@com_google_absl//absl/strings",
        "//mediapipe/framework/formats:rect_cc_proto",
        "//mediapipe/framework/port:ret_check",
        "//mediapipe/framework/formats:image_frame",
        "//mediapipe/framework/formats:matrix",
        "//mediapipe/framework/formats:image_frame_opencv",
        "//mediapipe/framework/port:opencv_core",
        "//mediapipe/framework/port:opencv_imgproc",
        "//mediapipe/framework/port:opencv_imgcodecs", 
        "//mediapipe/util:resource_util",
        ],
    alwayslink = 1,
)
```
**Note:** The file **poisson_blending_calculator** refers to our custom seamlesscloning calculator C++ file.

**H.** Add the following lines under cc_library section in the **opencv_linux.BUILD** file, inside the third_party directory.

`"lib/libopencv_photo.so",`
or
`"lib/x86_64-linux-gnu/libopencv_photo.so",`

**Note:** Make sure the file exists at the specified path in the system. This ensures that opencv can link the files for seamlesscloning from the module **photo** during build.

**Seamless Clone Custom Calculator**

A custom calculator can created by defining a new sub-class of the CalculatorBase class, implementing a number of methods, and registering the new sub-class with Mediapipe. At a minimum, a new calculator must implement the following methods - **GetContract, Open, Process and Close.**

The **Process** method continously takes inputs, processes them and produce outputs. We will write most of our code for seamless cloning within this method; whereas in **GetContract** we just specify the expected types of inputs and outputs of the calculator.

*Steps for seamless cloning:-*


1.  Convert the mask to binary Mat format with 0's representing background and 1 foreground.

2.  Resize the mask and background image to the size of the input image.

3.  Dilate the mask to include neighbouring background regions around borders.

4.  Find the largest contour and corresponding bounding rectangle from mask.

5.  Crop out the ROI from input and mask image using the bounding rectangle.

6.  Set the foreground pixels values of the mask to 255.

7.  Calculate the location of the center of the input roi image in the background.

8.  Perform seamless cloning of input on background, using mask and return the result.

Thus, the seamless clone calculator takes **three inputs** as CPU ImageFrame's i.e input image, background image and segmentaion mask. It produces a single ouput image frame,  representing the **blended image** in CPU. Finally, we save the results into a video file using **OpenCvVideoEncoderCalculator**.

Copy the calculator file '**[poisson_blending_calculator.cc](https://github.com/anilsathyan7/Portrait-Segmentation/blob/master/mediapipe_slimnet/desktop/poisson_blending_calculator.cc)**' into the directory - **mediapipe/calculators/image**.



**Build and Run**

To build the portrait_segmentation_gpu app on **desktop** using bazel, run:

In [ ]:
!bazel build -c opt --copt -DMESA_EGL_NO_X11_HEADERS --copt -DEGL_NO_X11 mediapipe/examples/desktop/portrait_segmentation:portrait_segmentation_gpu

Now, load two video files (i.e **portrait** and **background**) as inputs, run the application and save the output video.

In [ ]:
!GLOG_logtostderr=1 bazel-bin/mediapipe/examples/desktop/portrait_segmentation/portrait_segmentation_gpu --calculator_graph_config_file=/content/mediapipe/mediapipe/graphs/portrait_segmentation/portrait_segmentation_mobile_gpu.pbtxt --input_side_packets=side_video_path=/content/fire_vid.mp4,input_video_path=/content/grandma_vid.mp4,output_video_path=/content/output15.mp4

**Note:** If the run failed in a headless set-up(eg. google colab), then modify the file mediapipe/gpu/**gl_context_egl.cc** by removing the option '**EGL_WINDOW_BIT** and rebuild the application.